In [ ]:
import socket
import struct
from scipy.spatial.transform import Rotation as R
import numpy as np
import threading



def rot2rpy(Rt, degrees=False):
    """ Converts a rotation matrix into rpy / intrinsic xyz euler angle form.
    Args:    Rt (ndarray): The rotation matrix.
    Returns: ndarray: Angles in rpy / intrinsic xyz euler angle form.
    """
    return R.from_matrix(Rt).as_euler('xyz', degrees=degrees)


def rpy2rot(rpy, degrees=False):
    """Converts rpy / intrinsic xyz euler angles into a rotation matrix.
    Args:    rpy (array-like): rpy / intrinsic xyz euler angles.
             degrees (bool) : Whether or not euler_angles are in degrees (True), or radians (False).
    Returns: ndarray: The rotation matrix.
    """
    return R.from_euler('xyz', rpy, degrees=degrees).as_matrix()


def commandServer():
    #As server
    HOST = '192.168.0.102'#HOST = '127.0.0.1'  # Standard loopback interface address (localhost) 
    PORT = 65438        # Port to listen on (non-privileged ports are > 1023)


    #Server that sends commands
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s1:
        s1.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        s1.bind((HOST, PORT))
        s1.listen()

        #s.settimeout(0.1)
        conn, addr = s1.accept()
        with conn:
            print('Connected by', addr)
            s1.setblocking(False)
            inputstring=0
            while True:
                #while inputstring!='end':
                    inputstring=input("") #Press Enter to continue...
                    if (inputstring=='h' or inputstring=='j' or inputstring=='k' or inputstring=='l' or 
                        inputstring=='u' or inputstring=='o'or inputstring=='y' or inputstring=='i' or 
                        inputstring=='z' or inputstring=='x' or inputstring=='d' or inputstring=='c'or
                        inputstring=='home'):

                        data1=inputstring.encode('ascii')    
                        print(data1)
                        conn.send(data1)


                    if inputstring=='end': 
                        data1=b'end'
                        print(data1)
                        conn.send(data1) 
                        s1.shutdown(socket.SHUT_RDWR)
                        s1.close()
                        print("s1 connection shutdown and closed")
                        break

def poseClient():              
    #As client
    HOST2 = '192.168.0.103'
    PORT2= 65452   
    #Client that receives xyzRPY pose
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s2:
        s2.connect((HOST2, PORT2))
        #s.sendall(message)
        while True:
            data2 = s2.recv(64)  #48 bytes
            #print(data2)
            if data2==b'end':
                s2.shutdown(socket.SHUT_RDWR)
                s2.close()
                print("s2 connection shutdown and closed")
                break
            #value = struct.unpack('f',data2)
            unpacked = struct.unpack('ffffffffffffffff', data2)
            #print(unpacked)
            #print('Received', repr(data2))
            TransRotatmatrix=np.zeros([4,4])
            for i in range(4):
                TransRotatmatrix[i][:]=unpacked[0+(4*i):4+(4*i)]
            print(TransRotatmatrix)
            rpy=rot2rpy(TransRotatmatrix[0:3,0:3],True)
            x=TransRotatmatrix[0][3]
            y=TransRotatmatrix[1][3]
            z=TransRotatmatrix[2][3]
            roll=rpy[0]  #In degrees!
            pitch=rpy[1]
            yaw=rpy[2]
            print("x",x,"y",y,"z",z)
            print("roll",roll,"pitch",pitch,"yaw",yaw)
            
t1 = threading.Thread(target=poseClient, args=())
t2 = threading.Thread(target=commandServer, args=())

# starting thread 1
t1.start()
print("t1Start")
# starting thread 2
t2.start()
print("t2Start")
# wait until thread 1 is completely executed
t1.join()
# wait until thread 2 is completely executed
t2.join()

# both threads completely executed
print("Done!")        
         

t1Start
t2Start
[[-0.99934441 -0.00610222  0.03568637 -0.14532579]
 [-0.00426994  0.99867946  0.05119661 -0.30391228]
 [-0.03595166  0.05101066 -0.99805081  0.26136941]
 [ 0.          0.          0.          1.        ]]
x -0.14532579481601715 y -0.30391228199005127 z 0.26136940717697144
roll 177.07414215504588 pitch 2.060322291013616 yaw -179.75519145125745
[[-0.99934441 -0.00610222  0.03568637 -0.14532579]
 [-0.00426994  0.99867946  0.05119661 -0.30391228]
 [-0.03595166  0.05101066 -0.99805081  0.26136941]
 [ 0.          0.          0.          1.        ]]
x -0.14532579481601715 y -0.30391228199005127 z 0.26136940717697144
roll 177.07414215504588 pitch 2.060322291013616 yaw -179.75519145125745
s2 connection shutdown and closed


In [ ]:
from remote_FT_client import RemoteFTclient
from time import sleep

In [ ]:
client = RemoteFTclient( '192.168.0.103', 10000 )
print( client.prxy.system.listMethods() )

In [ ]:
print( "Get wrist force:" )

for i in range(5):
    print( '\t', client.get_wrist_force() )
    sleep( 0.020 )
    
print( "Bias wrist force" )
client.bias_wrist_force()
sleep( 3.0 )

for i in range(5):
    print( '\t', client.get_wrist_force() )
    sleep( 0.020 )